# Federated Learning with Flower project

Tutorials:
    
    https://github.com/OpenMined/PySyft/blob/dev/examples/tutorials/advanced/Federated%20Dataset.ipynb

In [8]:
import torch as th
import os
import numpy as np
import torchvision
import syft as sy

W0802 17:26:18.142209  1324 secure_random.py:22] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow (1.14.0). Fix this by compiling custom ops.


In [9]:
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torch import nn
import torch.nn.functional as F
from torch import optim
from collections import OrderedDict

%matplotlib inline

In [10]:
#creating a class with hyperparameters

class Arguments():
    def __init__(self):
        self.batch_size = 16
        self.test_batch_size = 16
        self.seed = 1


In [11]:
args = Arguments()

In [12]:
hook= sy.TorchHook(th)

In [13]:
#create workers
w1 = sy.VirtualWorker(hook, id="w1")
w2 = sy.VirtualWorker(hook, id="w2")
secure_worker3 = sy.VirtualWorker(hook, id="secure_worker3")

In [14]:
#share pointers between workers
w1.add_workers([w2, secure_worker3])
w2.add_workers([w1, secure_worker3])
secure_worker3.add_workers([w1, w2])

compute_nodes = [w1, w2]

W0802 17:26:18.856052  1324 base.py:624] Worker w2 already exists. Replacing old worker which could cause                     unexpected behavior
W0802 17:26:18.857048  1324 base.py:624] Worker secure_worker3 already exists. Replacing old worker which could cause                     unexpected behavior
W0802 17:26:18.858047  1324 base.py:624] Worker w1 already exists. Replacing old worker which could cause                     unexpected behavior
W0802 17:26:18.859071  1324 base.py:624] Worker secure_worker3 already exists. Replacing old worker which could cause                     unexpected behavior
W0802 17:26:18.860042  1324 base.py:624] Worker w1 already exists. Replacing old worker which could cause                     unexpected behavior
W0802 17:26:18.862036  1324 base.py:624] Worker w2 already exists. Replacing old worker which could cause                     unexpected behavior


In [15]:
data_dir = 'flower_data'
# number of subprocesses to use for data loading
#num_workers = 1
# how many samples per batch to load
#batch_size = 68
# percentage of training set to use as validation
#valid_size = 0.2

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])


# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + "\\train\\", transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '\\valid', transform=test_transforms)


#trainloader = th.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)
testloader = th.utils.data.DataLoader(test_data, batch_size=16)

In [39]:
import math
import logging
from torch.utils.data import Dataset

logger = logging.getLogger(__name__)

def dataset_federate(data_loader, workers):

    #Add a method to easily transform a torch.Dataset or a sy.BaseDataset
    #into a sy.FederatedDataset. The dataset given is split in len(workers)
    #part and sent to each workers
    logger.info("Scanning and sending data to {}...".format(", ".join([w.id for w in workers])))

    # take ceil to have exactly len(workers) sets after splitting
    data_size = math.ceil(len(train_data) / len(workers))
    print(len(train_data))

    # Fix for old versions of torchvision
#     if not hasattr(dataset, "data"):
#         if hasattr(dataset, "train_data"):
#             dataset.data = dataset.train_data
#         elif hasattr(dataset, "test_data"):
#             dataset.data = dataset.test_data
#         else:
#             raise AttributeError("Could not find inputs in dataset")
#     if not hasattr(dataset, "targets"):
#         if hasattr(dataset, "train_labels"):
#             dataset.targets = dataset.train_labels
#         elif hasattr(dataset, "test_labels"):
#             dataset.targets = dataset.test_labels
#         else:
#             raise AttributeError("Could not find targets in dataset")
    print(workers)
    dorkers=[0,0]
    datasets = []
    #data_loader = th.utils.data.DataLoader(dataset, batch_size=data_size, drop_last=True)
    for dataset_idx, (data, targets) in enumerate(data_loader):
        worker = workers[dataset_idx % len(workers)]
        print("Sending data to worker %s", worker.id)
        dorkers[dataset_idx%2]+=1
        data=th.tensor(data)
        data = data.send(worker)
        targets=th.tensor(targets) 
        targets = targets.send(worker)
        datasets.append(BaseDataset(data, targets))  # .send(worker)
    print(dorkers)
    logger.debug("Done!")
    return FederatedDataset(datasets)
  


In [ ]:
from torchvision import datasets
from syft.frameworks.torch.federated import FederatedDataset, FederatedDataLoader, BaseDataset
datasets.ImageFolder.federate = dataset_federate

federated_trainset = train_data.federate((w1,w2))

bob_trainset = federated_trainset.datasets['w1']
alice_trainset = federated_trainset.datasets['w2']

6552
(<VirtualWorker id:w1 #tensors:7948>, <VirtualWorker id:w2 #tensors:7946>)
Sending data to worker %s w1


C:\Users\France\temp-conda\envs\TFPrivateAI\lib\site-packages\syft\frameworks\torch\hook\hook.py:754: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  current_tensor = hook_self.torch.native_tensor(*args, **kwargs)


Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data t

Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data t

Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data t

Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data t

Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data t

Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data t

Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data t

Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data to worker %s w2
Sending data to worker %s w1
Sending data t

In [32]:
train_idx

3276

In [33]:
train_idx = int(len(train_data)/2)


In [34]:
# Dataset
#!dir flower_data\train

# Sending toy datasets to virtual workers
w1_train_dataset = sy.BaseDataset(train_data[:train_idx]).send(w1)
w2_train_dataset = sy.BaseDataset(train_data[train_idx:]).send(w2)


ValueError: too many values to unpack (expected 2)

In [15]:
w1_train_data = train_data[:train_idx]
w1_test_data = train_data[train_idx:]

ValueError: too many values to unpack (expected 2)